# Everything is markdown until: "Combining Data from April-mid June 2019"


## Challenge 1

import pandas as pd
data = pd.read_csv('turnstile_190504.csv')
data.head()

## Challenge 2

type(data['DATE'])

data['Date_Time'] = pd.to_datetime(data['DATE'], cache=True) + pd.to_timedelta(data['TIME'])

data.head()

data.dtypes

## Challenge 3

data.columns = data.columns.str.replace(' ', '')

data['Turnstile'] = data['C/A'] + data['UNIT'] + data['SCP'] + data['STATION']

entry_start = data.groupby(['DATE','Turnstile'], as_index = False).ENTRIES.min()

entry_end = data.groupby(['DATE','Turnstile'], as_index = False).ENTRIES.max()

traffic_entries = data.groupby(['DATE','Turnstile'], as_index = False).ENTRIES.agg(['max','min'])
traffic_exits = data.groupby(['DATE','Turnstile'], as_index = False).EXITS.agg(['max','min'])

traffic_entries['Daily Entries Traffic'] = traffic_entries['max'] - traffic_entries['min']

traffic_entries.head()

traffic_exits['Daily Exits Traffic'] = traffic_exits['max'] - traffic_exits['min']

traffic_exits.head()

data.iloc[:,-3]

grouped = data.groupby(['DATE','Turnstile'], as_index=False).agg({'ENTRIES': ['min', 'max'], 'EXITS': ['min', 'max']})

grouped.head()

grouped[('ENTRIES','Total_Entries')] = grouped[('ENTRIES','max')] - grouped[('ENTRIES', 'min')]

grouped[('EXITS','Total_Exits')] = grouped[('EXITS','max')] - grouped[('EXITS', 'min')]

grouped[('Total_Traffic','')] = grouped[('ENTRIES','Total_Entries')] + grouped[('EXITS', 'Total_Exits')]

grouped.head()

## Challenge 4

%matplotlib inline

import matplotlib.pyplot as plt

test = 'A002R05102-00-0059 ST'

test_df = grouped[grouped.iloc[:,1] == test]

grouped[('Date_Time','')] = pd.to_datetime(grouped[('DATE','')], cache=True)
grouped[('Week_Day','')] = grouped[('Date_Time','')].dt.weekday
grouped.head()

grouped[('Date_Time','')]

plt.figure(figsize=(10,3))
plt.plot(test_df[('DATE','')],test_df[('Total_Traffic','')])

## Challenge 6

data.head()

data.shape

station_entries = data.groupby(['DATE','STATION'], as_index = False).ENTRIES.agg(['max','min'])
station_exits = data.groupby(['DATE','STATION'], as_index = False).EXITS.agg(['max','min'])

station_entries['Daily Station Traffic'] = station_entries['max'] - station_entries['min']

station_exits['Daily Station Exits Traffic'] = station_exits['max'] - station_exits['min']

station_entries.head()

station_exits.head()

station_grouped = data.groupby(['DATE','STATION'], as_index=False).agg({'ENTRIES': ['min', 'max'], 'EXITS': ['min', 'max']})

station_grouped[('ENTRIES','Station_Entries')] = station_grouped[('ENTRIES','max')] - station_grouped[('ENTRIES', 'min')]
station_grouped[('EXITS','Station_Exits')] = station_grouped[('EXITS','max')] - station_grouped[('EXITS', 'min')]
station_grouped[('Total_Traffic','')] = station_grouped[('ENTRIES','Station_Entries')] + station_grouped[('EXITS', 'Station_Exits')]

station_grouped.head()

test_station = station_grouped[station_grouped.iloc[:,1] == '103 ST-CORONA']
station_grouped[('Date_Time','')] = pd.to_datetime(station_grouped[('DATE','')], cache=True)
station_grouped[('Week_Day','')] = station_grouped[('Date_Time','')].dt.weekday
station_grouped.head()
test_station.head()

test_station.sort_values(by=[('Week_Day','')], inplace = True)

## Challenge 7

plt.figure(figsize=(10,3))
plt.plot(test_station[('Week_Day','')],test_station[('Total_Traffic','')])

# Combining Data from April-mid June 2019


In [1]:
import numpy as np
import glob
import pandas as pd

%matplotlib inline

In [2]:
pwd

'/Users/patrickbovard/Documents/GitHubRepos/mta-project-1'

In [3]:
ls

CSVs/                         Rachel_Notebook.ipynb
Josef_MTA_Workbook.ipynb      Remote-Booth-Station.xls
MTA Project Submission.ipynb  StationList.xls
NYC_Map.png                   patrick_notebook.ipynb
Proj1.ipynb                   ritu_notebook.ipynb
Proj1_Ritu.ipynb              turnstile_190504.csv
README.md                     turnstile_190511.csv


In [4]:
cd CSVs

/Users/patrickbovard/Documents/GitHubRepos/mta-project-1/CSVs


In [5]:
ls

2019SKED.TXT          turnstile_190427.txt  turnstile_190525.txt
turnstile_190406.txt  turnstile_190504.txt  turnstile_190601.txt
turnstile_190413.txt  turnstile_190511.txt  turnstile_190608.txt
turnstile_190420.txt  turnstile_190518.txt  turnstile_190615.txt


In [6]:

extension = 'txt'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]


#combine all files in the list
combined_data = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
#data.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [7]:
combined_data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,00:00:00,REGULAR,7053483,2390505
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655


In [8]:
combined_data.shape

(2250808, 11)

In [9]:
combined_data.columns = combined_data.columns.str.replace(' ', '')
combined_data['Traffic'] = combined_data['ENTRIES'] + combined_data['EXITS']

In [10]:
station_traffic = combined_data.groupby(['STATION'], as_index=False).Traffic.sum()

In [11]:
station_traffic.head()

,STATION,Traffic
0,1 AV,1378353055191
1,103 ST,806847481771
2,103 ST-CORONA,67519250622
3,104 ST,3878362727391
4,110 ST,34617818646


In [12]:
station_traffic['STATION'].duplicated().sum()

0

In [13]:
#Station Name/Traffic for Citi Field (Mets)
station_traffic[station_traffic.STATION == 'METS-WILLETS PT']

,STATION,Traffic
278,METS-WILLETS PT,574777435664


In [14]:
#Station Name/Traffic for Yankee Stadium (Yankees)
station_traffic[station_traffic.STATION == '161/YANKEE STAD']

,STATION,Traffic
21,161/YANKEE STAD,1561851003244


In [15]:
station_traffic.head()

,STATION,Traffic
0,1 AV,1378353055191
1,103 ST,806847481771
2,103 ST-CORONA,67519250622
3,104 ST,3878362727391
4,110 ST,34617818646


In [16]:
combined_data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,00:00:00,REGULAR,7053483,2390505,9443988
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,04:00:00,REGULAR,7053511,2390508,9444019
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,08:00:00,REGULAR,7053534,2390536,9444070
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,12:00:00,REGULAR,7053618,2390596,9444214
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/11/2019,16:00:00,REGULAR,7053841,2390655,9444496


In [17]:
combined_data['STATION'][0:5].tolist()

['59 ST', '59 ST', '59 ST', '59 ST', '59 ST']

### Loading in 2019 MLB Schedule CSV

In [18]:
pwd

'/Users/patrickbovard/Documents/GitHubRepos/mta-project-1/CSVs'

In [19]:
schedule_2019 = pd.read_csv("2019SKED.TXT")

In [20]:
schedule_2019.head()

,20190320,0,Wed,SEA,AL,1,OAK,AL.1,1.1,n,Unnamed: 10,Unnamed: 11
0,20190321,0,Thu,SEA,AL,2,OAK,AL,2,n,NaN,NaN
1,20190328,0,Thu,ANA,AL,1,OAK,AL,3,d,NaN,NaN
2,20190328,0,Thu,ARI,NL,1,LAN,NL,1,d,NaN,NaN
3,20190328,0,Thu,ATL,NL,1,PHI,NL,1,d,NaN,NaN
4,20190328,0,Thu,BAL,AL,1,NYA,AL,1,d,NaN,NaN


Columns are the first row, need to rename.

In [21]:
schedule_2019.columns=['Date','GameNumber','WeekDay','VisitingTeam','VisitingTeamLeague','VisitingTeamGame','HomeTeam','HomeTeamLeague','HomeTeamGame','GameTime','CancelIndicator','MakeUpDate']

In [22]:
schedule_2019.head()

,Date,GameNumber,WeekDay,VisitingTeam,VisitingTeamLeague,VisitingTeamGame,HomeTeam,HomeTeamLeague,HomeTeamGame,GameTime,CancelIndicator,MakeUpDate
0,20190321,0,Thu,SEA,AL,2,OAK,AL,2,n,NaN,NaN
1,20190328,0,Thu,ANA,AL,1,OAK,AL,3,d,NaN,NaN
2,20190328,0,Thu,ARI,NL,1,LAN,NL,1,d,NaN,NaN
3,20190328,0,Thu,ATL,NL,1,PHI,NL,1,d,NaN,NaN
4,20190328,0,Thu,BAL,AL,1,NYA,AL,1,d,NaN,NaN


In [23]:
schedule_2019['DateTime'] = pd.to_datetime(schedule_2019.Date, format='%Y%m%d')

In [24]:
schedule_2019.head()

,Date,GameNumber,WeekDay,VisitingTeam,VisitingTeamLeague,VisitingTeamGame,HomeTeam,HomeTeamLeague,HomeTeamGame,GameTime,CancelIndicator,MakeUpDate,DateTime
0,20190321,0,Thu,SEA,AL,2,OAK,AL,2,n,NaN,NaN,2019-03-21
1,20190328,0,Thu,ANA,AL,1,OAK,AL,3,d,NaN,NaN,2019-03-28
2,20190328,0,Thu,ARI,NL,1,LAN,NL,1,d,NaN,NaN,2019-03-28
3,20190328,0,Thu,ATL,NL,1,PHI,NL,1,d,NaN,NaN,2019-03-28
4,20190328,0,Thu,BAL,AL,1,NYA,AL,1,d,NaN,NaN,2019-03-28


In [25]:
#Pulling the team abbreviations, each should have about 81 home games:
schedule_2019.groupby(['HomeTeam']).GameNumber.count()

HomeTeam
ANA    81
ARI    81
ATL    81
BAL    81
BOS    81
CHA    81
CHN    81
CIN    81
CLE    81
COL    81
DET    81
HOU    81
KCA    81
LAN    81
MIA    81
MIL    81
MIN    81
NYA    81
NYN    81
OAK    80
PHI    81
PIT    81
SDN    81
SEA    81
SFN    81
SLN    81
TBA    81
TEX    81
TOR    81
WAS    81
Name: GameNumber, dtype: int64

Finding New York Home games, with home teams NYA (Yankees) and NYN (Mets)

In [26]:
ny_home_games = ((schedule_2019.HomeTeam == 'NYA') |(schedule_2019.HomeTeam == 'NYN'))

In [27]:
ny_games_2019 = schedule_2019[ny_home_games]

In [28]:
ny_games_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 4 to 2414
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                162 non-null    int64         
 1   GameNumber          162 non-null    int64         
 2   WeekDay             162 non-null    object        
 3   VisitingTeam        162 non-null    object        
 4   VisitingTeamLeague  162 non-null    object        
 5   VisitingTeamGame    162 non-null    int64         
 6   HomeTeam            162 non-null    object        
 7   HomeTeamLeague      162 non-null    object        
 8   HomeTeamGame        162 non-null    int64         
 9   GameTime            162 non-null    object        
 10  CancelIndicator     6 non-null      object        
 11  MakeUpDate          6 non-null      object        
 12  DateTime            162 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(8)
memory 

Adding Date Time to ny_games dataframe:

In [29]:
ny_games_2019['DateTime'] = pd.to_datetime(ny_games_2019['Date'], format='%Y%m%d')

<ipython-input-29-05c8f9fc2885>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ny_games_2019['DateTime'] = pd.to_datetime(ny_games_2019['Date'], format='%Y%m%d')


In [30]:
ny_games_2019

,Date,GameNumber,WeekDay,VisitingTeam,VisitingTeamLeague,VisitingTeamGame,HomeTeam,HomeTeamLeague,HomeTeamGame,GameTime,CancelIndicator,MakeUpDate,DateTime
4,20190328,0,Thu,BAL,AL,1,NYA,AL,1,d,NaN,NaN,2019-03-28
27,20190330,0,Sat,BAL,AL,2,NYA,AL,2,d,NaN,NaN,2019-03-30
42,20190331,0,Sun,BAL,AL,3,NYA,AL,3,d,NaN,NaN,2019-03-31
61,20190401,0,Mon,DET,AL,5,NYA,AL,4,n,NaN,NaN,2019-04-01
72,20190402,0,Tue,DET,AL,6,NYA,AL,5,n,NaN,NaN,2019-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,20190925,0,Wed,MIA,NL,158,NYN,NL,158,n,NaN,NaN,2019-09-25
2379,20190926,0,Thu,MIA,NL,159,NYN,NL,159,n,NaN,NaN,2019-09-26
2384,20190927,0,Fri,ATL,NL,160,NYN,NL,160,n,NaN,NaN,2019-09-27
2399,20190928,0,Sat,ATL,NL,161,NYN,NL,161,n,NaN,NaN,2019-09-28


In [31]:
#Dropping columns that are not needed for merging the files
ny_games_2019.drop(columns = ['Date', 'GameNumber','VisitingTeam','VisitingTeamLeague','VisitingTeamGame', 'CancelIndicator','MakeUpDate'], inplace=True)

/Users/patrickbovard/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [32]:
ny_games_2019.head()

,WeekDay,HomeTeam,HomeTeamLeague,HomeTeamGame,GameTime,DateTime
4,Thu,NYA,AL,1,d,2019-03-28
27,Sat,NYA,AL,2,d,2019-03-30
42,Sun,NYA,AL,3,d,2019-03-31
61,Mon,NYA,AL,4,n,2019-04-01
72,Tue,NYA,AL,5,n,2019-04-02


In [33]:
ny_games_2019.groupby(['DateTime', 'HomeTeam']).HomeTeamGame.count().reset_index()

,DateTime,HomeTeam,HomeTeamGame
0,2019-03-28,NYA,1
1,2019-03-30,NYA,1
2,2019-03-31,NYA,1
3,2019-04-01,NYA,1
4,2019-04-02,NYA,1
...,...,...,...
157,2019-09-25,NYN,1
158,2019-09-26,NYN,1
159,2019-09-27,NYN,1
160,2019-09-28,NYN,1


Yankee Games only:

In [34]:
yankee_games = ny_games_2019[ny_games_2019.HomeTeam == 'NYA']
yankee_games.value_counts(yankee_games.HomeTeam)

HomeTeam
NYA    81
dtype: int64

Mets Games only:

In [35]:
mets_games = ny_games_2019[ny_games_2019.HomeTeam == 'NYN']
mets_games.value_counts(mets_games.HomeTeam)

HomeTeam
NYN    81
dtype: int64

Creating a dataframe of just the Mets/Yankee stops:

In [36]:
stadium_stations = combined_data[(combined_data.STATION == '161/YANKEE STAD') | (combined_data.STATION == 'METS-WILLETS PT')]

In [37]:
#Creating a DateTime object to help join to the subway data.
stadium_stations['DateTime'] = pd.to_datetime(stadium_stations['DATE'], cache=True)

<ipython-input-37-85dd7bdb2426>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stadium_stations['DateTime'] = pd.to_datetime(stadium_stations['DATE'], cache=True)


In [38]:
stadium_stations.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic,DateTime
72747,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,00:22:00,REGULAR,6029974,5954320,11984294,2019-05-11
72748,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,04:22:00,REGULAR,6029974,5954332,11984306,2019-05-11
72749,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,08:22:00,REGULAR,6030016,5954362,11984378,2019-05-11
72750,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,12:22:00,REGULAR,6030116,5954438,11984554,2019-05-11
72751,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,16:22:00,REGULAR,6030269,5954564,11984833,2019-05-11


Adding Games to the stadium_stations dataframe:

In [39]:
stadium_with_yankeegames = stadium_stations.merge(yankee_games, how='left', on='DateTime')
stadium_with_yankeegames.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic,DateTime,WeekDay,HomeTeam,HomeTeamLeague,HomeTeamGame,GameTime
0,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,00:22:00,REGULAR,6029974,5954320,11984294,2019-05-11,NaN,NaN,NaN,NaN,NaN
1,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,04:22:00,REGULAR,6029974,5954332,11984306,2019-05-11,NaN,NaN,NaN,NaN,NaN
2,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,08:22:00,REGULAR,6030016,5954362,11984378,2019-05-11,NaN,NaN,NaN,NaN,NaN
3,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,12:22:00,REGULAR,6030116,5954438,11984554,2019-05-11,NaN,NaN,NaN,NaN,NaN
4,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,16:22:00,REGULAR,6030269,5954564,11984833,2019-05-11,NaN,NaN,NaN,NaN,NaN


In [40]:
stadium_with_yankeegames.drop(columns=['WeekDay','HomeTeamLeague','HomeTeamGame','GameTime'], inplace=True)
stadium_with_yankeegames.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic,DateTime,HomeTeam
0,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,00:22:00,REGULAR,6029974,5954320,11984294,2019-05-11,NaN
1,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,04:22:00,REGULAR,6029974,5954332,11984306,2019-05-11,NaN
2,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,08:22:00,REGULAR,6030016,5954362,11984378,2019-05-11,NaN
3,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,12:22:00,REGULAR,6030116,5954438,11984554,2019-05-11,NaN
4,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,16:22:00,REGULAR,6030269,5954564,11984833,2019-05-11,NaN


In [41]:
stadium_with_yankeegames.rename(columns = {'HomeTeam':'YankeeGame'}, inplace=True)
stadium_with_yankeegames.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic,DateTime,YankeeGame
0,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,00:22:00,REGULAR,6029974,5954320,11984294,2019-05-11,NaN
1,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,04:22:00,REGULAR,6029974,5954332,11984306,2019-05-11,NaN
2,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,08:22:00,REGULAR,6030016,5954362,11984378,2019-05-11,NaN
3,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,12:22:00,REGULAR,6030116,5954438,11984554,2019-05-11,NaN
4,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,16:22:00,REGULAR,6030269,5954564,11984833,2019-05-11,NaN


Adding Mets Games and combining:

In [42]:
stadium_with_combined_games = stadium_with_yankeegames.merge(mets_games, how='left', on='DateTime')
stadium_with_combined_games.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic,DateTime,YankeeGame,WeekDay,HomeTeam,HomeTeamLeague,HomeTeamGame,GameTime
0,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,00:22:00,REGULAR,6029974,5954320,11984294,2019-05-11,NaN,Sat,NYN,NL,39.0,n
1,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,04:22:00,REGULAR,6029974,5954332,11984306,2019-05-11,NaN,Sat,NYN,NL,39.0,n
2,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,08:22:00,REGULAR,6030016,5954362,11984378,2019-05-11,NaN,Sat,NYN,NL,39.0,n
3,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,12:22:00,REGULAR,6030116,5954438,11984554,2019-05-11,NaN,Sat,NYN,NL,39.0,n
4,N203,R195,00-00-00,161/YANKEE STAD,BD4,IND,05/11/2019,16:22:00,REGULAR,6030269,5954564,11984833,2019-05-11,NaN,Sat,NYN,NL,39.0,n


In [43]:
stadium_with_combined_games.drop(columns=['WeekDay','HomeTeamLeague','HomeTeamGame','GameTime'], inplace=True)
stadium_with_combined_games.rename(columns = {'HomeTeam':'MetsGame'}, inplace=True)

In [44]:
stadium_with_combined_games.value_counts(stadium_with_combined_games.YankeeGame)

YankeeGame
NYA    15813
dtype: int64

In [45]:
stadium_with_combined_games.value_counts(stadium_with_combined_games.MetsGame)

MetsGame
NYN    14622
dtype: int64

Removing na values with NoGame:

In [46]:
stadium_with_combined_games['YankeeGame'].fillna('NoGame', inplace=True)

In [47]:
stadium_with_combined_games['MetsGame'].fillna('NoGame', inplace=True)

In [48]:
#Combined Sheet with Mets/Yankee Games, pulling a sample to audit with mlb.com:
stadium_with_combined_games.sample(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,Traffic,DateTime,YankeeGame,MetsGame
27981,N203,R195,00-03-00,161/YANKEE STAD,BD4,IND,05/05/2019,16:22:00,REGULAR,5389681,2110797,7500478,2019-05-05,NYA,NoGame
33378,R532H,R328,02-00-04,METS-WILLETS PT,7,IRT,05/03/2019,17:00:00,REGULAR,433003,1272802,1705805,2019-05-03,NYA,NoGame
21022,R532H,R328,02-00-03,METS-WILLETS PT,7,IRT,06/14/2019,13:00:00,REGULAR,541145,1261998,1803143,2019-06-14,NoGame,NYN
16375,R262,R195,03-03-00,161/YANKEE STAD,4BD,IRT,05/31/2019,00:22:00,REGULAR,2327469,1300912,3628381,2019-05-31,NYA,NoGame
14629,R532,R328,00-06-04,METS-WILLETS PT,7,IRT,04/26/2019,01:00:00,REGULAR,125128,13390,138518,2019-04-26,NoGame,NYN
20078,R262B,R195,05-00-05,161/YANKEE STAD,4BD,IRT,06/08/2019,04:22:00,REGULAR,236583,515471,752054,2019-06-08,NoGame,NYN
21782,N203,R195,00-00-04,161/YANKEE STAD,BD4,IND,03/30/2019,08:22:00,REGULAR,10957194,10250753,21207947,2019-03-30,NYA,NoGame
12454,N203,R195,00-00-02,161/YANKEE STAD,BD4,IND,04/21/2019,12:22:00,REGULAR,4267960,1669280,5937240,2019-04-21,NYA,NoGame
9195,R532H,R328,02-05-01,METS-WILLETS PT,7,IRT,04/17/2019,05:00:00,REGULAR,1,513,514,2019-04-17,NYA,NoGame
27090,R532H,R328,02-00-02,METS-WILLETS PT,7,IRT,04/08/2019,21:00:00,REGULAR,128254,280090,408344,2019-04-08,NoGame,NoGame


All dates in sample set match with schedule correctly

Setting masks for games, no games to use for filtering:

In [49]:
yankee_game_mask = (stadium_with_combined_games.YankeeGame == 'NYA')

In [50]:
non_yankee_game_mask = (stadium_with_combined_games.YankeeGame == 'NoGame')

In [51]:
mets_game_mask = (stadium_with_combined_games.MetsGame == 'NYN')

In [52]:
non_mets_game_mask = (stadium_with_combined_games.MetsGame == 'NoGame')